In [2]:
from transformers import AutoModelForImageClassification, AutoImageProcessor

repo_name = "JacobJan/swin-tiny-patch4-window7-224-finetuned-eurosat"

image_processor = AutoImageProcessor.from_pretrained(repo_name)
model = AutoModelForImageClassification.from_pretrained(repo_name)

In [7]:
# prepare image for the model
from PIL import Image
image = Image.open('test_dataset_gray/dressoir_mccp.jpg')
# encoding = image_processor(image.convert("RGB"), return_tensors="pt")
# print(encoding.pixel_values.shape)

In [4]:
import torch

# forward pass
with torch.no_grad():
    outputs = model(**encoding)
    logits = outputs.logits

In [5]:
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

Predicted class: 0


In [16]:
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights

img = read_image("scraped_images/2386_K 1644_id=cp131969_linz.jpg")

# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)

# Step 4: Use the model and print the predicted category
prediction = model(batch).squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")

cloak: 25.4%
